# First approach, simple neural networks

In [1]:
import pickle
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from graph import *
from node import *

from sklearn.model_selection import train_test_split
from evaluation import MyPredictionModel, evaluate, load_model, EdgePredictor, OmarPredictionModel, evaluate_all_metrics


In [2]:
with open('./data/graphs.dat', 'rb') as file:
    train_graphs_list: List[Graph] = pickle.load(file)
    train_graphs_list, test_graphs = train_test_split(train_graphs_list, test_size=0.2, random_state=42)
all_part_ids = []
all_family_ids = []
for graph in train_graphs_list:
    for n in graph.get_nodes():
        all_part_ids.append(int(n.get_part().get_part_id()))
        all_family_ids.append(int(n.get_part().get_family_id()))

part_vocab_size = max(all_part_ids) + 1
family_vocab_size = max(all_family_ids) + 1
print(f"Part Vocab Size: {part_vocab_size}")
print(f"Family Vocab Size: {family_vocab_size}")

Part Vocab Size: 2271
Family Vocab Size: 96


In [3]:
############################################################
# 2) Build a PyTorch Dataset
############################################################
class LinkPredictionDataset(Dataset):

    """
    Creates positive/negative samples from each Graph.
    For each Graph:
      - Collect all nodes
      - For every pair (i, j), check if it's an edge (label=1) or not (label=0)
    """
    def __init__(self, graphs: List[Graph]):
        super().__init__()
        self.samples = []

        for g in graphs:
            # Get the nodes and edges
            node_list = g.get_nodes()          # List[Node]
            edge_list = g.get_edges()          # List of (Node, Node)
            edge_set = self.get_edge_list(edge_list)        # for quick membership checks

            # Map node ID -> (part_id, family_id)
            node_id_to_features = {}
            for node in node_list:
                node_id_to_features[node.get_id()] = (
                    node.get_part().get_part_id(),
                    node.get_part().get_family_id()
                )

            # We'll gather all node IDs from the node list
            node_ids = [n.get_id() for n in node_list]
            id_to_node = {n.get_id(): n for n in node_list}

            # Create all (i, j) pairs
            for i in node_ids:
                for j in node_ids:
                    if i == j:
                        continue
                    part_i, fam_i = node_id_to_features[i]
                    part_j, fam_j = node_id_to_features[j]

                    # Sort the pair for an undirected edge check
                    pair = tuple(sorted([id_to_node[i], id_to_node[j]],
                                        key=lambda x: x.get_id()))
                    label = 1 if pair in edge_set else 0

                    self.samples.append((int(part_i), int(fam_i), int(part_j), int(fam_j), int(label)))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        return self.samples[idx]  # (part_i, fam_i, part_j, fam_j, label)
    def get_edge_list(self, __edges: Dict[Node, List[Node]]):
        edge_pairs = set()  # use a set to avoid duplicates

        for src, neighbors in __edges.items():
            for dst in neighbors:
                # Sort the pair so that (NodeA, NodeB) == (NodeB, NodeA)
                sorted_pair = tuple(sorted([src, dst], key=lambda n: n.get_id()))
                edge_pairs.add(sorted_pair)

        return edge_pairs  # Now we have a list of (Node, Node) pairs


def train_edge_predictor(model, train_graphs_list, optimizer, criterion, epochs=100):
    model.train()
    for epoch in range(epochs):
        total_loss = 0.0
        for batch in dataloader:
            part_i, fam_i, part_j, fam_j, label = batch
            # Convert to Long / Float for embeddings + BCE
            part_i = part_i.long()
            fam_i  = fam_i.long()
            part_j = part_j.long()
            fam_j  = fam_j.long()
            label  = label.float()

            optimizer.zero_grad()
            logits = model(part_i, fam_i, part_j, fam_j)
            loss = criterion(logits, label)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        avg_loss = total_loss / len(dataloader)
        print(f"Epoch {epoch+1}/{epochs} - Loss: {avg_loss:.4f}")



### Now train it and write it to disk

In [4]:
# Create the dataset and dataloader
dataset = LinkPredictionDataset(train_graphs_list)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

# Create the model, criterion, and optimizer
model_EdgePredictor = EdgePredictor(part_vocab_size, family_vocab_size, embed_dim=16, hidden_dim=32)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model_EdgePredictor.parameters(), lr=0.001)

# train the model
train_edge_predictor(model_EdgePredictor, train_graphs_list, optimizer, criterion, epochs=50)
torch.save(model_EdgePredictor.state_dict(), "model_EdgePredictor.pth")


Epoch 1/50 - Loss: 0.1575
Epoch 2/50 - Loss: 0.0923
Epoch 3/50 - Loss: 0.0860
Epoch 4/50 - Loss: 0.0835
Epoch 5/50 - Loss: 0.0820
Epoch 6/50 - Loss: 0.0811
Epoch 7/50 - Loss: 0.0803
Epoch 8/50 - Loss: 0.0798
Epoch 9/50 - Loss: 0.0793
Epoch 10/50 - Loss: 0.0788
Epoch 11/50 - Loss: 0.0784
Epoch 12/50 - Loss: 0.0781
Epoch 13/50 - Loss: 0.0780
Epoch 14/50 - Loss: 0.0777
Epoch 15/50 - Loss: 0.0775
Epoch 16/50 - Loss: 0.0772
Epoch 17/50 - Loss: 0.0772
Epoch 18/50 - Loss: 0.0769
Epoch 19/50 - Loss: 0.0768
Epoch 20/50 - Loss: 0.0767
Epoch 21/50 - Loss: 0.0764
Epoch 22/50 - Loss: 0.0764
Epoch 23/50 - Loss: 0.0763
Epoch 24/50 - Loss: 0.0761
Epoch 25/50 - Loss: 0.0761
Epoch 26/50 - Loss: 0.0760
Epoch 27/50 - Loss: 0.0760
Epoch 28/50 - Loss: 0.0759
Epoch 29/50 - Loss: 0.0758
Epoch 30/50 - Loss: 0.0757
Epoch 31/50 - Loss: 0.0757
Epoch 32/50 - Loss: 0.0755
Epoch 33/50 - Loss: 0.0756
Epoch 34/50 - Loss: 0.0755
Epoch 35/50 - Loss: 0.0754
Epoch 36/50 - Loss: 0.0753
Epoch 37/50 - Loss: 0.0753
Epoch 38/5

Now evaluate it
We developed our own metrics:

Evaluation Metrics:

1. Precision, Recall, F1-score:
   - Precision: Measures the proportion of correctly predicted edges out of all predicted edges.
   - Recall: Measures the proportion of correctly predicted edges out of all actual edges.
   - F1-score: Harmonic mean of precision and recall, balancing both metrics.

2. Hamming Distance:
   - Counts the number of differing edges between the predicted and target adjacency matrices.

3. Jaccard Similarity:
   - Measures the overlap between predicted and actual edge sets, calculated as the intersection over the union.

4. Graph Edit Distance:
   - Computes the minimum number of edge insertions, deletions, or substitutions to transform the predicted graph into the target graph.



We took the mean over all test samples.



In [5]:

model_file_path = 'model_EdgePredictor.pth'
prediction_model: MyPredictionModel = load_model(model_file_path)


instances = [(graph.get_parts(), graph) for graph in test_graphs[:500]]

evaluate_all_metrics(prediction_model, instances)



/Users/omarbouattour/PycharmProjects/ai-project/evaluation.py:190: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  loaded_model.load_state_dict(torch.load(file_path, map_locat

Evaluation Results:
  Precision: 0.8328
  Recall: 1.0000
  F1-score: 0.8983
  Hamming Distance: 4.3600
  Jaccard Similarity: 0.8328
  Graph Edit Distance: 2.1800


The evaluate method was provided. It calculates the edge accuracy over multiple permutations.

In [6]:
eval_score = evaluate(prediction_model, instances)
print(eval_score)

93.16962056914629


# Second Method: GNN

In [7]:
def train_graph_predictor(model, train_graphs_list, optimizer, criterion, epochs=100):
    model.train()
    for epoch in range(epochs):
        print("EPOCH:", epoch)
        total_loss = 0.0

        for graph in train_graphs_list:
            optimizer.zero_grad()

            # Sort nodes
            nodes = sorted(
                graph.get_nodes(),
                key=lambda node: (node.get_part().get_part_id(), node.get_part().get_family_id())
            )

            # Prepare part/family IDs
            part_ids = torch.tensor(
                [int(node.get_part().get_part_id()) for node in nodes],
                dtype=torch.long
            )
            family_ids = torch.tensor(
                [int(node.get_part().get_family_id()) for node in nodes],
                dtype=torch.long
            )

            # Build adjacency on the same device
            part_order = tuple(node.get_part() for node in nodes)
            adjacency_matrix = torch.tensor(
                graph.get_adjacency_matrix(part_order),
                dtype=torch.float32
            )

            # Forward pass
            logits = model(part_ids, family_ids)

            # Flatten for loss
            target = adjacency_matrix.flatten()
            loss = criterion(logits.flatten(), target)

            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        avg_loss = total_loss / len(train_graphs_list)
        print(f"Epoch {epoch+1}/{epochs} - Loss: {avg_loss:.4f}")


### Now train it and write it to disk

In [8]:
model = OmarPredictionModel(part_vocab_size, family_vocab_size, embed_dim=1, gnn_hidden_dim=32)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.BCEWithLogitsLoss()

train_graph_predictor(model, train_graphs_list, optimizer, criterion, epochs=100)
torch.save(model.state_dict(), "graph_predictor_model.pth")



EPOCH: 0
Epoch 1/100 - Loss: 0.2602
EPOCH: 1
Epoch 2/100 - Loss: 0.1575
EPOCH: 2
Epoch 3/100 - Loss: 0.1307
EPOCH: 3
Epoch 4/100 - Loss: 0.1170
EPOCH: 4
Epoch 5/100 - Loss: 0.1077
EPOCH: 5
Epoch 6/100 - Loss: 0.1015
EPOCH: 6
Epoch 7/100 - Loss: 0.0972
EPOCH: 7
Epoch 8/100 - Loss: 0.0941
EPOCH: 8
Epoch 9/100 - Loss: 0.0916
EPOCH: 9
Epoch 10/100 - Loss: 0.0896
EPOCH: 10
Epoch 11/100 - Loss: 0.0879
EPOCH: 11
Epoch 12/100 - Loss: 0.0863
EPOCH: 12
Epoch 13/100 - Loss: 0.0854
EPOCH: 13
Epoch 14/100 - Loss: 0.0845
EPOCH: 14
Epoch 15/100 - Loss: 0.0837
EPOCH: 15
Epoch 16/100 - Loss: 0.0830
EPOCH: 16
Epoch 17/100 - Loss: 0.0823
EPOCH: 17
Epoch 18/100 - Loss: 0.0814
EPOCH: 18
Epoch 19/100 - Loss: 0.0809
EPOCH: 19
Epoch 20/100 - Loss: 0.0802
EPOCH: 20
Epoch 21/100 - Loss: 0.0797
EPOCH: 21
Epoch 22/100 - Loss: 0.0791
EPOCH: 22
Epoch 23/100 - Loss: 0.0787
EPOCH: 23
Epoch 24/100 - Loss: 0.0783
EPOCH: 24
Epoch 25/100 - Loss: 0.0780
EPOCH: 25
Epoch 26/100 - Loss: 0.0779
EPOCH: 26
Epoch 27/100 - Loss: 

### Now evaluate it

In [9]:
model_file_path = 'graph_predictor_model.pth'
prediction_model: MyPredictionModel = load_model(model_file_path)


instances = [(graph.get_parts(), graph) for graph in test_graphs[:500]]
eval_score = evaluate(prediction_model, instances)
print(eval_score)


tensor([[1.9649e-09, 9.9815e-01, 2.7526e-05, 9.8669e-01, 9.8669e-01, 5.6983e-01,
         1.0000e+00],
        [9.9886e-01, 4.0073e-07, 9.7829e-01, 1.4960e-11, 1.4960e-11, 9.2606e-01,
         4.2307e-22],
        [6.2928e-05, 9.5864e-01, 2.1852e-09, 1.0881e-05, 1.0881e-05, 7.9660e-06,
         1.3441e-11],
        [9.8483e-01, 1.6359e-11, 1.1631e-05, 1.7812e-27, 1.7812e-27, 4.0445e-10,
         1.5226e-27],
        [9.8483e-01, 1.6359e-11, 1.1631e-05, 1.7812e-27, 1.7812e-27, 4.0445e-10,
         1.5226e-27],
        [5.5733e-01, 9.2849e-01, 8.6539e-06, 6.5135e-10, 6.5135e-10, 1.4919e-03,
         1.0555e-02],
        [1.0000e+00, 1.1076e-20, 7.0999e-13, 6.2422e-28, 6.2422e-28, 5.3689e-03,
         0.0000e+00]])
tensor([[3.3435e-27, 3.3435e-27, 4.2218e-01, 1.0017e-12, 9.9493e-01, 1.1459e-16,
         1.1459e-16],
        [3.3435e-27, 3.3435e-27, 4.2218e-01, 1.0017e-12, 9.9493e-01, 1.1459e-16,
         1.1459e-16],
        [4.3787e-01, 4.3787e-01, 9.3608e-10, 1.6276e-02, 9.0890e-06, 1.1

/Users/omarbouattour/PycharmProjects/ai-project/evaluation.py:200: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  loaded_model.load_state_dict(torch.load(file_path, map_locat

tensor([[2.1664e-08, 9.9985e-01, 1.2173e-06, 1.2173e-06, 9.8259e-09],
        [9.9977e-01, 1.0484e-25, 1.3979e-01, 1.3979e-01, 1.0000e+00],
        [2.2252e-06, 1.4123e-01, 3.3435e-27, 3.3435e-27, 2.7908e-13],
        [2.2252e-06, 1.4123e-01, 3.3435e-27, 3.3435e-27, 2.7908e-13],
        [3.7795e-09, 1.0000e+00, 2.4787e-13, 2.4787e-13, 5.1536e-30]])
tensor([[2.1664e-08, 1.3653e-08, 1.7626e-05, 9.9985e-01, 1.8476e-06],
        [1.1674e-08, 6.1656e-09, 3.0757e-05, 9.2765e-05, 5.5200e-01],
        [2.4121e-05, 2.1492e-05, 7.0381e-16, 1.0000e+00, 1.2509e-14],
        [9.9981e-01, 9.7105e-05, 1.0000e+00, 1.9668e-19, 9.9562e-01],
        [3.3183e-06, 5.5333e-01, 2.0630e-14, 9.9572e-01, 7.9160e-27]])
tensor([[2.1664e-08, 2.1664e-08, 9.9990e-01, 1.0700e-05, 7.3213e-09, 7.3213e-09,
         1.8476e-06, 1.8476e-06],
        [2.1664e-08, 2.1664e-08, 9.9990e-01, 1.0700e-05, 7.3213e-09, 7.3213e-09,
         1.8476e-06, 1.8476e-06],
        [9.9986e-01, 9.9986e-01, 1.5550e-20, 9.8739e-01, 1.6751e-04,

Again we evaluate on our metrics

In [10]:
evaluate_all_metrics(prediction_model, instances)

tensor([[1.9649e-09, 9.9815e-01, 2.7526e-05, 9.8669e-01, 9.8669e-01, 5.6983e-01,
         1.0000e+00],
        [9.9886e-01, 4.0073e-07, 9.7829e-01, 1.4960e-11, 1.4960e-11, 9.2606e-01,
         4.2307e-22],
        [6.2928e-05, 9.5864e-01, 2.1852e-09, 1.0881e-05, 1.0881e-05, 7.9660e-06,
         1.3441e-11],
        [9.8483e-01, 1.6359e-11, 1.1631e-05, 1.7812e-27, 1.7812e-27, 4.0445e-10,
         1.5226e-27],
        [9.8483e-01, 1.6359e-11, 1.1631e-05, 1.7812e-27, 1.7812e-27, 4.0445e-10,
         1.5226e-27],
        [5.5733e-01, 9.2849e-01, 8.6539e-06, 6.5135e-10, 6.5135e-10, 1.4919e-03,
         1.0555e-02],
        [1.0000e+00, 1.1076e-20, 7.0999e-13, 6.2422e-28, 6.2422e-28, 5.3689e-03,
         0.0000e+00]])
tensor([[3.3435e-27, 3.3435e-27, 4.2218e-01, 1.0017e-12, 9.9493e-01, 1.1459e-16,
         1.1459e-16],
        [3.3435e-27, 3.3435e-27, 4.2218e-01, 1.0017e-12, 9.9493e-01, 1.1459e-16,
         1.1459e-16],
        [4.3787e-01, 4.3787e-01, 9.3608e-10, 1.6276e-02, 9.0890e-06, 1.1